In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

# Load the dataset from your Google Drive
import pandas as pd
file_path = '/content/drive/My Drive/MachineLearning/transactions.csv'  # adjust this path
df = pd.read_csv(file_path)

# Show first few rows
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,8,CASH_OUT,158007.12,C424875646,0.00,0.00,C1298177219,474016.32,1618631.97,0
1,236,CASH_OUT,457948.30,C1342616552,0.00,0.00,C1323169990,2720411.37,3178359.67,0
2,37,CASH_IN,153602.99,C900876541,11160428.67,11314031.67,C608741097,3274930.56,3121327.56,0
3,331,CASH_OUT,49555.14,C177696810,10865.00,0.00,C462716348,0.00,49555.14,0
4,250,CASH_OUT,29648.02,C788941490,0.00,0.00,C1971700992,56933.09,86581.10,0


In [5]:
# Load the dataset from your Google Drive
import pandas as pd
file_path = '/content/drive/My Drive/MachineLearning/transactions.csv'  # adjust this path
df = pd.read_csv(file_path)

# Show first few rows
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,8,CASH_OUT,158007.12,C424875646,0.00,0.00,C1298177219,474016.32,1618631.97,0
1,236,CASH_OUT,457948.30,C1342616552,0.00,0.00,C1323169990,2720411.37,3178359.67,0
2,37,CASH_IN,153602.99,C900876541,11160428.67,11314031.67,C608741097,3274930.56,3121327.56,0
3,331,CASH_OUT,49555.14,C177696810,10865.00,0.00,C462716348,0.00,49555.14,0
4,250,CASH_OUT,29648.02,C788941490,0.00,0.00,C1971700992,56933.09,86581.10,0


In [6]:
# Explore dataset structure
print("Data Info:")
print(df.info())

# Summary statistics
print("\nData Description:")
print(df.describe())

# Check missing values
print("\nMissing Values:")
print(df.isnull().sum())

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   step            199999 non-null  int64  
 1   type            199999 non-null  object 
 2   amount          199999 non-null  float64
 3   nameOrig        199999 non-null  object 
 4   oldbalanceOrg   199999 non-null  float64
 5   newbalanceOrig  199999 non-null  float64
 6   nameDest        199999 non-null  object 
 7   oldbalanceDest  199999 non-null  float64
 8   newbalanceDest  199999 non-null  float64
 9   isFraud         199999 non-null  int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 15.3+ MB
None

Data Description:
                step        amount  oldbalanceOrg  newbalanceOrig  \
count  199999.000000  1.999990e+05   1.999990e+05    1.999990e+05   
mean      243.289836  1.802425e+05   8.314361e+05    8.523333e+05   
std       141.800473  6.255482e+

In [15]:
# Fill missing numerical values with mean
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
imputer_num = SimpleImputer(strategy='mean')
df[num_cols] = imputer_num.fit_transform(df[num_cols])

# Fill missing categorical values with mode
cat_cols = df.select_dtypes(include=['object']).columns
imputer_cat = SimpleImputer(strategy='most_frequent')
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])

print("\nMissing values after imputation:")
print(df.isnull().sum())


Missing values after imputation:
step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
dtype: int64


In [18]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd


# Label encode ordinal features (e.g., payment method if there's order)
if 'payment_method' in df.columns:
    le = LabelEncoder()
    df['payment_method'] = le.fit_transform(df['payment_method'])

# One-Hot encode nominal features
# Identify nominal columns excluding 'nameOrig', 'nameDest' and the target column
nominal_cols = df.select_dtypes(include='object').columns.tolist()
target_column = 'isFraud' if 'isFraud' in df.columns else None

if target_column and target_column in nominal_cols:
  nominal_cols.remove(target_column)

# Exclude 'nameOrig', 'nameDest' from nominal columns as they are identifiers
nominal_cols = [col for col in nominal_cols if col not in ['nameOrig', 'nameDest']]

df = pd.get_dummies(df, columns=nominal_cols, drop_first=True)


# ------------------------------
# 4. Scale numerical features

# Choose columns to scale (all numerical columns except 'isFraud' if it exists)
scale_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
if target_column and target_column in scale_cols:
  scale_cols.remove(target_column)


# StandardScaler (good if outliers)
scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])

# ------------------------------
# 5. Train-Test Split

# Define target and features
if target_column and target_column in df.columns:
    X = df.drop(target_column, axis=1)
    y = df[target_column]
else:
    X = df.copy()
    y = None

# Split only if target is defined
if y is not None:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("\nTrain/Test shapes:")
    print("X_train:", X_train.shape)
    print("X_test:", X_test.shape)
    print("y_train:", y_train.shape)
    print("y_test:", y_test.shape)
else:
    print("\nNo target column found; skipping train-test split.")

# ------------------------------
# 6. Final dataset
print("\nPreprocessed dataset preview:")
print(df.head())


Train/Test shapes:
X_train: (159999, 12)
X_test: (40000, 12)
y_train: (159999,)
y_test: (40000,)

Preprocessed dataset preview:
       step    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0 -1.659306 -0.035545   C424875646      -0.288462       -0.292161   
1 -0.051409  0.443941  C1342616552      -0.288462       -0.292161   
2 -1.454793 -0.042586   C900876541       3.583585        3.586034   
3  0.618548 -0.208917   C177696810      -0.284692       -0.292161   
4  0.047321 -0.240741   C788941490      -0.288462       -0.292161   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  type_CASH_OUT  \
0  C1298177219       -0.187603        0.110208      0.0           True   
1  C1323169990        0.492531        0.540219      0.0           True   
2   C608741097        0.660422        0.524496      0.0          False   
3   C462716348       -0.331119       -0.322380      0.0           True   
4  C1971700992       -0.313882       -0.312172      0.0           True   

   type_DEB